In [1]:
import ee
import geemap.core as geemap

In [2]:
# Authenticate and initialize Google Earth Engine API
ee.Authenticate()

# Connect to my project number
ee.Initialize(project='716069062206')

In [ ]:
# Import other packages used in the tutorial
%matplotlib inline
import geemap
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, chi2

from pprint import pprint 

# this part is taken from A.. Zuspan: https://developers.google.com/earth-engine/tutorials/community/pseudo-invariant-feature-matching
# code was modified and extended according to the needs of the diploma thesis

In [4]:
# Display an image in a one percent linear stretch
def display_ls(image, map, name, centered = False):
    bns = image.bandNames().length().getInfo()
    if bns == 3:
        image = image.rename('B1', 'B2', 'B3')
        pb_99 = ['B1_p99', 'B2_p99', 'B3_p99']
        pb_1 = ['B1_p1', 'B2_p1', 'B3_p1']
        img = ee.Image.rgb(image.select('B1'), image.select('B2'), image.select('B3'))
    else:
        image = image.rename('B1')
        pb_99 = ['B1_p99']
        pb_1 = ['B1_p1']
        img = image.select('B1')
    percentiles = image.reduceRegion(ee.Reducer.percentile([1, 99]), maxPixels=1e11)
    mx = percentiles.values(pb_99)
    if centered:
        mn = ee.Array(mx).multiply(-1).toList()
    else:
        mn = percentiles.values(pb_1)
    map.addLayer(img, {'min': mn, 'max': mx}, name)

In [5]:
aoi = ee.Geometry.Polygon(
        [[[14.673153981451712,49.82721862997025],
          [15.2767153828189,49.82721862997025],
          [15.2767153828189,50.08873547414839],
          [14.673153981451712,50.08873547414839]]]) # study area

In [6]:
def collect(aoi):
    try:
        bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
        im1 = ee.Image( ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') # LC08_191025_20160804 - landsat 9
                  .filter(ee.Filter.eq('system:index', 'LC09_191025_20230917'))
                  .first()
                  .clip(aoi)
                  .select(bands))
        im2 = ee.Image( ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') # landsat 5
                  .filter(ee.Filter.eq('system:index', 'LT05_191025_20000909'))
                  .first()
                  .clip(aoi)
                  .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).rename(bands) )
        timestamp = im1.date().format('E MMM dd HH:mm:ss YYYY')
        print(timestamp.getInfo())
        timestamp = im2.date().format('E MMM dd HH:mm:ss YYYY')
        print(timestamp.getInfo())
        return (im1, im2)
    except Exception as e:
        print('Error: %s'%e) # print the error message

im1, im2 = collect(aoi)

Sun Sep 17 09:51:04 2023
Sat Sep 09 09:29:10 2000


In [7]:
# Interactive map
M1 = geemap.Map()
M1.centerObject(aoi, 11)


In [8]:
visirbands = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6', 'SR_B7']
visbands = ['SR_B4','SR_B3','SR_B2']

diff = im1.subtract(im2).select(visbands)
display_ls(im1.select(visbands), M1, 'Landsat 9')
display_ls(im2.select(visbands), M1, 'Landsat 5')
display_ls(diff, M1, 'Difference')
M1

Map(center=[49.95825012032068, 14.974934682135093], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
reference = im1.select(visirbands) # reference image - L9
target = im2.select(visirbands) # target image - L5
# control proj
print(f'Reference has same CRS as target: {reference.projection().getInfo() == target.projection().getInfo()} \nReprojecting:')
# Parametry pro Collection 2 Surface Reflectance
SCALE  = 0.0000275
OFFSET = -0.2


def to_sr_full(image, bands):
    sr = ( image.select(bands)
              .multiply(SCALE)
              .add(OFFSET)
              .rename(bands) )
    
    out = image.addBands(sr, bands, overwrite=True)
    return out

reference = to_sr_full(reference, visirbands)
target    = to_sr_full(target,    visirbands)

reference_new = reference.reproject(crs=reference.projection(), scale=reference.projection().nominalScale())
reference_new = reference_new.clip(aoi)
crs_info_reference_n = reference_new.projection().getInfo()
target = target.reproject(crs=reference.projection(), scale=reference.projection().nominalScale())
target = target.clip(aoi)
crs_info_target = target.projection().getInfo()
# check if the projections are now the same
print(f'Reference has same CRS as target: {crs_info_reference_n == crs_info_target }')

Reference has same CRS as target: False 
Reprojecting:
Reference has same CRS as target: True


In [10]:
# calculate spectral distance as a mesure of changee between the images
method = 'EMD' # try SID, SAM, EMD
distance = target.spectralDistance(reference_new, method).rename('distance') # you can try different metric: sid, sam, sed, emd

threshold = distance.reduceRegion(
    reducer= ee.Reducer.percentile([10]), # 20, 30, 40, 50, 60, 70
    geometry= aoi,
    scale= 30,
    bestEffort= True,
    maxPixels= 1e13,
).getNumber('distance') 
print(f'Threshold value for PIF pixels: {threshold.getInfo()} \nCalculated by {method} method') # threshold value for pseudo invariant feature

pif = distance.lt(threshold).rename('pif') # pif - pseudo invariant feature

Threshold value for PIF pixels: 0.08428391090389135 
Calculated by EMD method


In [11]:
pif

In [ ]:
methods = ['SAM'] #'SID', 'SAM'
percentiles = [70] # 20, 30, 40, 50, 60, 70

pif_masks = {}

for method in methods:
    # count spectral distance
    distance = target.spectralDistance(reference_new, method).rename('distance')

    for p in percentiles:
        # threshold for the given percentile
        threshold = distance.reduceRegion(
            reducer=ee.Reducer.percentile([p]),
            geometry=aoi,
            scale=30,
            bestEffort=True,
            maxPixels=1e13
        ).getNumber('distance')

        pif = distance.lt(threshold).rename('pif'+f'_{method}_P{p}')
        pif_masks[f'{method}_P{p}'] = pif
        # 
        print(f'Threshold for {method} at {p}th percentile:', threshold.getInfo())

Threshold for SAM at 70th percentile: 0.23828209410207649


In [13]:
for key, img in pif_masks.items():
    bands = img.bandNames().getInfo()
    print(f'{key}: {bands}')

SAM_P70: ['pif_SAM_P70']


In [ ]:
# functions calculating linear regresion for each band
def match_band(band):
    band = ee.String(band)
    pif = pif_masks['SAM_P70']
    
    before_pif = target.select([band]).updateMask(pif)
    after_pif = reference_new.select([band]).updateMask(pif)
    
    combined = ee.Image.cat([
        after_pif.rename('y'),  
        before_pif.rename('x')   
    ])
    
    # calculate linear regression
    coeffs = combined.reduceRegion(
        reducer=ee.Reducer.linearFit(),
        geometry=aoi,
        scale=30,
        maxPixels=1e12,
        bestEffort=False
    )

    # normalize L5 image based on calculated coeffs
    corrected = target.select([band]) \
        .multiply(ee.Number(coeffs.get('scale'))) \
        .add(ee.Number(coeffs.get('offset')))
    return corrected

def get_band_coeffs(band):
    '''description...'''
    band = ee.String(band)
    pif = pif_masks['SAM_P70']

    before_pif = target.select([band]).updateMask(pif)
    after_pif = reference_new.select([band]).updateMask(pif)

    coeffs = ee.Image.cat([after_pif, before_pif]).reduceRegion(
        reducer=ee.Reducer.linearFit(),
        geometry=aoi,
        scale=30,
        maxPixels=1e6,
        bestEffort=True
    )

    return ee.Dictionary({
        'band': band,
        'scale': coeffs.get('scale'),
        'offset': coeffs.get('offset')
    })

def band_coefs_before(band):
    band = ee.String(band)

    before_pif = target.select([band])
    after_pif = reference_new.select([band])

    coeffs = ee.Image.cat([after_pif, before_pif]).reduceRegion(
        reducer=ee.Reducer.linearFit(),
        geometry=aoi,
        scale=30,
        maxPixels=1e6,
        bestEffort=True
    )

    return ee.Dictionary({
        'band': band,
        'scale': coeffs.get('scale'),
        'offset': coeffs.get('offset')
    })

In [15]:
bands = ee.List([ee.String('SR_B2'), ee.String('SR_B3'), ee.String('SR_B4'), ee.String('SR_B5'), ee.String('SR_B6'), ee.String('SR_B7')]); 

print(f'PIF normalization band by band....\n')
matched_bands = bands.map(match_band)

# Calculate and print coefficients
coeffs_list_before = bands.map(band_coefs_before)
coeffs_list_before = coeffs_list_before.getInfo()

print("Calculated coefficients for each band before normalization:")
for coeff in coeffs_list_before:
    band = coeff['band']
    scale = round(coeff['scale'], 4)
    offset = round(coeff['offset'], 4)
    print(f"{band}: scale = {scale}, offset = {offset}")

coeffs_list_after = bands.map(get_band_coeffs)
coeffs_list_after = coeffs_list_after.getInfo()
print("\nCalculated coefficients for each band after normalization")
for coeff in coeffs_list_after:
    band = coeff['band']
    scale = round(coeff['scale'], 4)
    offset = round(coeff['offset'], 4)
    print(f"{band}: scale = {scale}, offset = {offset}")

# match norm bands into one
matched_image = ee.ImageCollection(matched_bands).toBands().rename(bands)
#matched_image = ee.Image.cat(matched_bands)#.rename(bands)
matched_image = matched_image.clip(aoi)

# reproject no_change_mask
noChangeMask = pif.reproject(crs=reference.projection(), scale=reference.projection().nominalScale())
noChangeMask = noChangeMask.clip(aoi)

# rename bands
bands_pif = ee.List([ee.String('pif_B2'), ee.String('pif_B3'), ee.String('pif_B4'), ee.String('pif_B5'), ee.String('pif_B6'), ee.String('SR_B7')])

l5_pif = matched_image.select(bands).rename(bands_pif)
l9 = reference_new
l5 = target

# reproject l9 to l9 crs
l9 = l9.reproject(crs = reference.projection(), scale=reference.projection().nominalScale())
l5 = l5.reproject(crs = reference.projection(), scale=reference.projection().nominalScale())
l5_pif = l5_pif.reproject(crs = reference.projection(), scale=reference.projection().nominalScale())

# import srtm model
srtm = ee.Image('USGS/SRTMGL1_003').clip(aoi) # SRTM DEM
print(f'\nSRTM model for AOI imported.\n')

# reproject to the same projection as the reference image
srtm_repr = srtm.reproject(crs=reference.projection(), scale=reference.projection().nominalScale())
srtm_repr = srtm_repr.clip(aoi)
crs_info_srtm = srtm_repr.projection().getInfo()
print(f'SRTM band CRS equal to other bands: {crs_info_srtm == l5.projection().getInfo()}')

# Create slope and aspect from SRTM data
print(f'Calculating slope and aspect...')
slope = ee.Terrain.slope(srtm_repr).clip(aoi) # slope
aspect = ee.Terrain.aspect(srtm_repr).clip(aoi) # aspect
crs_info_slope = slope.projection().getInfo()
crs_info_aspect = aspect.projection().getInfo()
print(f'Slope, aspect bands CRS equal to other bands: {crs_info_slope == l5.projection().getInfo()}, {crs_info_aspect == l5.projection().getInfo()}\n')

PIF normalization band by band....

Calculated coefficients for each band before normalization:
SR_B2: scale = 0.5644, offset = 0.0235
SR_B3: scale = 0.6773, offset = 0.0274
SR_B4: scale = 0.5971, offset = 0.0377
SR_B5: scale = 0.2549, offset = 0.1588
SR_B6: scale = 0.733, offset = 0.0523
SR_B7: scale = 0.648, offset = 0.0496

Calculated coefficients for each band after normalization
SR_B2: scale = 0.7574, offset = 0.0126
SR_B3: scale = 0.8052, offset = 0.0159
SR_B4: scale = 0.8035, offset = 0.0168
SR_B5: scale = 0.5667, offset = 0.0799
SR_B6: scale = 0.8625, offset = 0.0193
SR_B7: scale = 0.8678, offset = 0.0113

SRTM model for AOI imported.

SRTM band CRS equal to other bands: True
Calculating slope and aspect...
Slope, aspect bands CRS equal to other bands: True, True



In [16]:
# add layer for no-change mask
noChangeMask_inv = noChangeMask.updateMask(noChangeMask).clip(aoi) # mask out no-change pixels
M1.addLayer(noChangeMask_inv, {'palette': ['red']}, 'no-change mask', True)

M1

Map(bottom=178204.0, center=[49.95825012032068, 14.974934682135093], controls=(WidgetControl(options=['positio…

In [ ]:
# count of no-change pixels
px_count = noChangeMask_inv.reduceRegion(
    reducer   = ee.Reducer.count(),
    geometry  = aoi,
    scale     = 30, 
    maxPixels = 1e9,
    bestEffort=True
)

print('No-change pixels count:', px_count.get('pif_SAM_P10').getInfo()) # name has to corespond to the same method and percentile

EEException: Dictionary.get: Dictionary does not contain key: 'pif_SAM_P10'.

In [126]:
# save coeffs before and after normalization to a csv file
import pandas as pd
# create a DataFrame from the list of dictionaries
df_before = pd.DataFrame(coeffs_list_before)
df_before.rename(columns={'scale': 'scale_before', 'offset': 'offset_before'}, inplace=True)
df_after = pd.DataFrame(coeffs_list_after)
df_after.rename(columns={'scale': 'scale_after', 'offset': 'offset_after'}, inplace=True)
# merge the two DataFrames on the 'band' column
merged_df = pd.merge(df_before, df_after, on='band')
# save the merged DataFrame to a CSV file
merged_df.to_csv('PIF_coeffs_SAM_70.csv', index=False) # # PIF_coeffs_SID, PIF_coeffs_SAM, PIF_coeffs_EMD
#merged_df

In [18]:
# create function for mask clouds
def mask_clouds(image):
    '''description...'''
    qa = image.select('QA_PIXEL')
    cloud_shadow_bit_mask = 1 << 3
    clouds_bit_mask = 1 << 5
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0) \
        .And(qa.bitwiseAnd(clouds_bit_mask).eq(0))
    return image.updateMask(mask)

# prepare ndvi and ndmi index calculation
def ndvi9(image):
    '''Calculate NDVI index'''
    return image.expression('(NIR - RED) / (NIR + RED)', {
        'NIR': image.select('SR_B5'), 
        'RED': image.select('SR_B4')  
    }).rename('NDVI9')

def ndmi9(image):
    '''Calculate NDMI index'''
    return image.expression('(NIR - SWIR) / (NIR + SWIR)', {
        'NIR': image.select('SR_B5'), 
        'SWIR': image.select('SR_B6')  
    }).rename('NDMI9')

def ndvi5(image):
    return image.expression('(NIR - RED) / (NIR + RED)', {
        'NIR': image.select('SR_B4'), 
        'RED': image.select('SR_B3')  
    }).rename('NDVI5')

def ndmi5(image):
    return image.expression('(NIR - SWIR) / (NIR + SWIR)', {
        'NIR': image.select('SR_B4'), 
        'SWIR': image.select('SR_B5')  
    }).rename('NDMI5')

# caluclaten NDVI and NDMI VARIANCE
ndvi_variance9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2023-04-01', '2023-10-01') \
    .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
    .map(mask_clouds) \
    .map(ndvi9) \
    .select('NDVI9') \
    .reduce(ee.Reducer.variance()) \
    .rename('NDVI_variance9') \
    .reproject(crs=reference.projection(), scale=reference.projection().nominalScale()) \
    .clip(aoi) # NDVI variance for LANDSAT 9  

ndmi_variance9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2023-04-01', '2023-10-01') \
    .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
    .map(mask_clouds) \
    .map(ndmi9) \
    .select('NDMI9') \
    .reduce(ee.Reducer.variance()) \
    .rename('NDMI_variance9') \
    .reproject(crs=reference.projection(), scale=reference.projection().nominalScale()) \
    .clip(aoi) # NDMI variance for LANDSAT 9  

ndvi_variance5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2000-04-01', '2000-10-01') \
    .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
    .map(mask_clouds) \
    .map(ndvi5) \
    .select('NDVI5') \
    .reduce(ee.Reducer.variance()) \
    .rename('NDVI_variance5') \
    .reproject(crs=reference.projection(), scale=reference.projection().nominalScale()) \
    .clip(aoi) # NDVI variance for LANDSAT 5

ndmi_variance5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2000-04-01', '2000-10-01') \
    .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
    .map(mask_clouds) \
    .map(ndmi5) \
    .select('NDMI5') \
    .reduce(ee.Reducer.variance()) \
    .rename('NDMI_variance5') \
    .reproject(crs=reference.projection(), scale=reference.projection().nominalScale()) \
    .clip(aoi) # NDMI variance for LANDSAT 5

print(f'NDVI_var9, NDMI_var9, NDVI_var5, NDMI_var5 CRS equal to other bands:')
print(ndvi_variance9.projection().getInfo() == l5.projection().getInfo()) # check if the projections are the same
print(ndmi_variance9.projection().getInfo() == l5.projection().getInfo())
print(ndvi_variance5.projection().getInfo() == l9.projection().getInfo())
print(ndmi_variance5.projection().getInfo() == l9.projection().getInfo())

NDVI_var9, NDMI_var9, NDVI_var5, NDMI_var5 CRS equal to other bands:
True
True
True
True


In [19]:
# Pro Landsat 5 (target)
l5_ndvi = ndvi5(l5).reproject(crs=reference.projection(), scale=reference.projection().nominalScale())
l5_ndmi = ndmi5(l5).reproject(crs=reference.projection(), scale=reference.projection().nominalScale())

# Pro Landsat 9 (reference_new)
l9_ndvi = ndvi9(l9).reproject(crs=reference.projection(), scale=reference.projection().nominalScale())
l9_ndmi = ndmi9(l9).reproject(crs=reference.projection(), scale=reference.projection().nominalScale())

In [20]:
# rename bands
#bands_pif = ee.List([ee.String('pif_B2'), ee.String('pif_B3'), ee.String('pif_B4'), ee.String('pif_B5'), ee.String('pif_B6')])
bands_l5 = ee.List([ee.String('l5_B2'), ee.String('l5_B3'), ee.String('l5_B4'), ee.String('l5_B5'), ee.String('l5_B6'), ee.String('l5_B7')])
bands_l9 = ee.List([ee.String('l9_B2'), ee.String('l9_B3'), ee.String('l9_B4'), ee.String('l9_B5'), ee.String('l9_B6'), ee.String('l9_B7')])

#l5_pif = matched_image.select(bands).rename(bands_pif)
l9 = l9.select(bands).rename(bands_l9)
l5 = l5.select(bands).rename(bands_l5)

# reproject l9 to l9 crs
l9 = l9.reproject(crs = reference.projection(), scale=reference.projection().nominalScale())
l5 = l5.reproject(crs = reference.projection(), scale=reference.projection().nominalScale())
l5_pif = l5_pif.reproject(crs = reference.projection(), scale=reference.projection().nominalScale())

In [21]:
# merge all bands into one
l9 = l9.clip(aoi)
all_bands = l9.cat(l9.clip(aoi), l5.clip(aoi), l5_pif.clip(aoi), noChangeMask.clip(aoi)) # 
all_bands_srtm = all_bands.addBands(srtm_repr).addBands(slope).addBands(aspect)
all_bands_srtm_var = all_bands_srtm.addBands(ndvi_variance9).addBands(ndmi_variance9).addBands(ndvi_variance5).addBands(ndmi_variance5)
all_bands_srtm_var_ind = all_bands_srtm_var.addBands(l9_ndvi).addBands(l9_ndmi).addBands(l5_ndvi).addBands(l5_ndmi)
all_bands_reproj = all_bands_srtm_var_ind.reproject(crs=reference.projection(), scale=reference.projection().nominalScale())
all_bands_fl = all_bands_reproj.toFloat().setDefaultProjection(crs=reference.projection())

In [22]:
all_bands_fl

In [23]:
vis_pif = ['pif_B4','pif_B3','pif_B2']
vis_params_pif = {
    'bands': ['pif_B4','pif_B3','pif_B2'],  # R-G-B
    'min':   0,
    'max':   0.2,
    'gamma': 0.9
}
#display_ls(all_bands_fl.select(vis_pif), M1, 'PIF')
M1.addLayer(all_bands_fl, vis_params_pif, 'PIF – RGB')
M1

Map(bottom=178204.0, center=[49.95825012032068, 14.974934682135093], controls=(WidgetControl(options=['positio…

In [25]:
all_bands_fl = all_bands_fl.clip(aoi)
# Export to Google Drive
driveexport = ee.batch.Export.image.toDrive(all_bands_fl,
                        description='pif_emd_out_5',
                        folder='EngineExports',
                        region=aoi,
                        crs='EPSG:32633',
                        fileNamePrefix='PIF_EMD_5',
                        fileFormat='GeoTIFF',
                        scale=30, maxPixels=1e12)
driveexport.start()
print('Exporting iMAD to Google Drive\n task id: %s'%str(driveexport.id))

Exporting iMAD to Google Drive
 task id: GBCYNFRRBKSXEWRCDTSZZKZS
